In [ ]:
# pip install bitsandbytes transformers accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer
from huggingface_hub import login
from IPython.display import Markdown

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
login("token")


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",     # NormalFloat4 (better accuracy)
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer)

# Load model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

In [ ]:
inputs = tokenizer("Explain quantum computing to a 12 boards student", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100 , streamer=streamer)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(Markdown(response))